In [ ]:
import sys
sys.path.append('/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP')
import yaml
import numpy as np
from torch import Tensor
import torch
import matplotlib.pyplot as plt
from run_experiments.prepare_dataset import *
from utils_general import prepare_common_background_info, pred4all_outputs_inputs, evaluate_on_single_output, plot_traindata_testdata_fittedgp, neg_log_likelihood, normalised_mean_square_error
from code_blocks.our_models.lvmogp_svi import LVMOGP_SVI
from code_blocks.likelihoods.gaussian_likelihood import GaussianLikelihood

In [ ]:
torch.set_default_dtype(torch.float64)

In [ ]:
config_name = '/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/configs/exchange/Scale_RBF/lvmogp_unfix.yaml'
model_path = '/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/experiments_results/exchange/Scale_RBF/lvmogp_unfix/2024-03-02_21:29:20/model.pth'
likelihood_path = '/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/experiments_results/exchange/Scale_RBF/lvmogp_unfix/2024-03-02_21:29:20/likelihood.pth'

## Load in model

In [ ]:
with open(config_name) as file:
    config = yaml.safe_load(file)

my_model = LVMOGP_SVI(
        n_outputs = config['n_outputs'],
        n_input = config['n_input_train'],                    # NOTE PAY ATTENTION, not total n_inputs.
        input_dim = config['input_dim'],
        latent_dim = config['latent_dim'],
        n_inducing_input = config['n_inducing_input'],
        n_inducing_latent = config['n_inducing_latent'],
        learn_inducing_locations_latent = config['learn_inducing_locations_latent'],
        learn_inducing_locations_input = config['learn_inducing_locations_input'],
        latent_kernel_type = config['latent_kernel_type'],
        input_kernel_type = config['input_kernel_type'],
        trainable_latent_dim = config['trainable_latent_dim'] if 'trainable_latent_dim' in config else None,
        latent_first_init = None,                               # if None, random initialization
        latent_second_init = None,                              # if None, random initialization
        NNEncoder = config['NNEncoder'],
        layers = None                                           # if none, adopt default value [4, 8, 4]
    )   

my_likelihood = GaussianLikelihood()

In [ ]:
model_state_dict = torch.load(model_path)
my_model.load_state_dict(model_state_dict)

likelihood_state_dict = torch.load(likelihood_path)
my_likelihood.load_state_dict(likelihood_state_dict)

## Load in dataset

In [ ]:
if config['dataset_type'] == 'exchange':
    data_inputs, data_Y_squeezed, ls_of_ls_train_input, ls_of_ls_test_input, train_sample_idx_ls, test_sample_idx_ls, means, stds = prepare_exchange_data(config)
        
n_data4visual = 500
inputs_total4visual = Tensor(np.linspace(2007, 2008, n_data4visual))

## Look at latent variables

## Test via Integration

In [ ]:
common_background_information = prepare_common_background_info(my_model, config)

### explore three ways to compute inverse of cholesky factor of K_uu

In [ ]:
from linear_operator.operators import KroneckerProductLinearOperator, TriangularLinearOperator, LinearOperator, CholLinearOperator
from linear_operator.utils.cholesky import psd_safe_cholesky
import numpy as np
from tqdm import trange
from linear_operator import to_dense
from gpytorch.settings import _linalg_dtype_cholesky

def _cholesky_factor(induc_induc_covar: LinearOperator) -> TriangularLinearOperator:
        L = psd_safe_cholesky(to_dense(induc_induc_covar).type(_linalg_dtype_cholesky.value()), max_tries=4)
        return TriangularLinearOperator(L)

In [ ]:
K_uu_latent = common_background_information['K_uu_latent'].to_dense()
K_uu_input = common_background_information['K_uu_input'].to_dense()

In [ ]:
# K_uu_input[K_uu_input < 1e-4] = 0.

In [ ]:
K_uu = KroneckerProductLinearOperator(K_uu_latent, K_uu_input).to_dense()
K_uu_inv = torch.linalg.solve(K_uu, torch.eye(K_uu.size(-1)))

In [ ]:
torch.allclose(torch.kron(K_uu_latent, K_uu_input), K_uu)

In [ ]:
diff1 = torch.eye(K_uu.size(-1)) - (K_uu @ K_uu_inv)
diff2 = torch.eye(K_uu.size(-1)) - (K_uu_inv @ K_uu)

print(diff1.abs().max())
print(diff2.abs().max())

#### Way 1: first cholesky of each component, then inverse for each of them, finally kronecker product

In [ ]:
chol_K_uu_latent = _cholesky_factor(K_uu_latent)
chol_K_uu_input = _cholesky_factor(K_uu_input)
chol_K_uu_latent_inv = torch.linalg.solve(chol_K_uu_latent, torch.eye(K_uu_latent.size(-1)))
chol_K_uu_input_inv = torch.linalg.solve(chol_K_uu_input, torch.eye(K_uu_input.size(-1)))

# chol_K_uu_inv_1_torch = torch.kron(chol_K_uu_latent_inv, chol_K_uu_input_inv)

In [ ]:
diff3_1 = torch.eye(chol_K_uu_latent.size(-1)) - (chol_K_uu_latent_inv @ chol_K_uu_latent)
diff3_2 = torch.eye(chol_K_uu_latent.size(-1)) - (chol_K_uu_latent @ chol_K_uu_latent_inv)

diff4_1 = torch.eye(chol_K_uu_input.size(-1)) - (chol_K_uu_input_inv @ chol_K_uu_input)
diff4_2 = torch.eye(chol_K_uu_input.size(-1)) - (chol_K_uu_input @ chol_K_uu_input_inv)

print(diff3_1.abs().max())
print(diff3_2.abs().max())
print(diff4_1.abs().max())
print(diff4_2.abs().max())

In [ ]:
chol_K_uu_inv_1 = KroneckerProductLinearOperator(
            chol_K_uu_latent_inv, chol_K_uu_input_inv) # .to_dense()

In [ ]:
K_uu_inv_via_chol_1 = chol_K_uu_inv_1._transpose_nonbatch() @ chol_K_uu_inv_1

In [ ]:
diff7_1 = torch.eye(K_uu_inv_via_chol_1.size(-1)) - K_uu_inv_via_chol_1 @ K_uu
diff7_2 = torch.eye(K_uu_inv_via_chol_1.size(-1)) - K_uu @ K_uu_inv_via_chol_1

print(diff7_1.abs().max())
print(diff7_2.abs().max())

#### Way2: first kronecker product, then apply cholesky factor, finally inverse

In [ ]:
chol_K_uu = _cholesky_factor(K_uu).to_dense()
chol_K_uu_inv_2 = torch.linalg.solve(chol_K_uu, torch.eye(K_uu.size(-1)))

In [ ]:
K_uu_inv_via_chol_2 = chol_K_uu_inv_2.T @ chol_K_uu_inv_2

In [ ]:
diff5_1 = torch.eye(chol_K_uu.size(-1)) - chol_K_uu_inv_2 @ chol_K_uu
diff5_2 = torch.eye(chol_K_uu.size(-1)) - chol_K_uu @ chol_K_uu_inv_2

print(diff5_1.abs().max())
print(diff5_2.abs().max())

In [ ]:
diff6_1 = torch.eye(K_uu_inv_via_chol_2.size(-1)) - K_uu_inv_via_chol_2 @ K_uu
diff6_2 = torch.eye(K_uu_inv_via_chol_2.size(-1)) - K_uu @ K_uu_inv_via_chol_2

print(diff6_1.abs().max())
print(diff6_2.abs().max())

#### Way3: first kronecker product, then inverse, finally cholesky factor

In [ ]:
chol_K_uu_inv_3 = _cholesky_factor(K_uu_inv).to_dense()

### prediction

In [ ]:
all_pred_mean, all_pred_var = pred4all_outputs_inputs(my_model=my_model,
                                                        my_likelihood=my_likelihood,
                                                        data_inputs=data_inputs,
                                                        config=config,
                                                        common_background_information=common_background_information,
                                                        approach='integration')

In [ ]:
all_pred_mean4visual, all_pred_var4visual = pred4all_outputs_inputs(my_model=my_model,
                                                        my_likelihood=my_likelihood,
                                                        data_inputs=inputs_total4visual,
                                                        config=config,
                                                        common_background_information=common_background_information,
                                                        approach='integration',
                                                        not4visual=False,
                                                        n_data4visual=n_data4visual)


In [ ]:
train_data_predict = all_pred_mean[train_sample_idx_ls]
train_rmse = (train_data_predict - data_Y_squeezed[train_sample_idx_ls]).square().mean().sqrt()
print('Global Train RMSE via integration', train_rmse)

w_test_data_predict = all_pred_mean[test_sample_idx_ls]
test_rmse = (w_test_data_predict - data_Y_squeezed[test_sample_idx_ls]).square().mean().sqrt()
print('Global Test RMSE via integration', test_rmse)


In [ ]:
train_nll = neg_log_likelihood(Target=data_Y_squeezed[train_sample_idx_ls], GaussianMean=all_pred_mean[train_sample_idx_ls], GaussianVar=all_pred_var[train_sample_idx_ls])
test_nll = neg_log_likelihood(Target=data_Y_squeezed[test_sample_idx_ls], GaussianMean=all_pred_mean[test_sample_idx_ls], GaussianVar=all_pred_var[test_sample_idx_ls])

print('Global Train negative log likelihood via integration:', train_nll)
print('Global Test negative log likelihood via integration:', test_nll)

In [ ]:
function_index = 12
w_train_input, w_train_target, w_test_input, w_test_target, w_gp_pred_mean, w_gp_pred_std, performance_dirct = evaluate_on_single_output(
                                                        function_index = function_index,
                                                        data_inputs = data_inputs,
                                                        data_Y_squeezed = data_Y_squeezed, 
                                                        ls_of_ls_train_input = ls_of_ls_train_input,
                                                        ls_of_ls_test_input = ls_of_ls_test_input,
                                                        train_sample_idx_ls = train_sample_idx_ls,
                                                        test_sample_idx_ls = test_sample_idx_ls,
                                                        all_pred_mean = all_pred_mean,
                                                        all_pred_var = all_pred_var,
                                                        n_data4visual = n_data4visual,
                                                        all_pred_mean4visual = all_pred_mean4visual,
                                                        all_pred_var4visual = all_pred_var4visual                                                        
                                                        )

plot_traindata_testdata_fittedgp(train_X=w_train_input, train_Y=w_train_target, test_X=w_test_input, test_Y=w_test_target, gp_X=inputs_total4visual, gp_pred_mean=w_gp_pred_mean, gp_pred_std=w_gp_pred_std, inducing_points_X=my_model.variational_strategy.inducing_points_input.data, n_inducing_C=config['n_inducing_input']) # NOTE: input is C not X